# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

,a,b,g,c,d,e,f
5,96.987234,-0.965396,1234,96.021839,97.952630,192.043677,195.905260
6,351.425314,-0.158075,1234,351.267239,351.583389,702.534478,703.166778
7,586.456579,0.757985,1234,587.214563,585.698594,1174.429127,1171.397188
8,555.060109,0.032453,1234,555.092563,555.027656,1110.185125,1110.055312
9,885.814954,0.684928,1234,886.499883,885.130026,1772.999766,1770.260052
10,436.631393,0.415573,1234,437.046966,436.215820,874.093932,872.431641
11,594.437721,0.516486,1234,594.954207,593.921235,1189.908414,1187.842471
12,79.539730,0.820537,1234,80.360267,78.719193,160.720534,157.438387
13,624.591227,-0.373867,1234,624.217359,624.965094,1248.434718,1249.930188
14,542.729757,-0.508303,1234,542.221454,543.238059,1084.442908,1086.476119


In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

,objective_c,objective_d
5,-96.021839,97.952630
6,-351.267239,351.583389
7,-587.214563,585.698594
8,-555.092563,555.027656
9,-886.499883,885.130026
10,-437.046966,436.215820
11,-594.954207,593.921235
12,-80.360267,78.719193
13,-624.217359,624.965094
14,-542.221454,543.238059


In [13]:
# This is also available as a method
vocs.objective_data(data)

,objective_c,objective_d
5,-96.021839,97.952630
6,-351.267239,351.583389
7,-587.214563,585.698594
8,-555.092563,555.027656
9,-886.499883,885.130026
10,-437.046966,436.215820
11,-594.954207,593.921235
12,-80.360267,78.719193
13,-624.217359,624.965094
14,-542.221454,543.238059


In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[ -96.0218387 ,   97.9526299 ],
       [-351.26723903,  351.58338881],
       [-587.2145633 ,  585.69859425],
       [-555.09256271,  555.02765625],
       [-886.49988295,  885.13002605],
       [-437.04696617,  436.21582035],
       [-594.95420685,  593.92123535],
       [ -80.36026708,   78.71919347],
       [-624.21735923,  624.96509404],
       [-542.22145375,  543.23805934]])

In [15]:
vocs.constraint_data(data)

,constraint_e,constraint_f
5,190.043677,-195.905260
6,700.534478,-703.166778
7,1172.429127,-1171.397188
8,1108.185125,-1110.055312
9,1770.999766,-1770.260052
10,872.093932,-872.431641
11,1187.908414,-1187.842471
12,158.720534,-157.438387
13,1246.434718,-1249.930188
14,1082.442908,-1086.476119


In [16]:
vocs.feasibility_data(data)

,feasible_e,feasible_f,feasible
5,False,True,False
6,False,True,False
7,False,True,False
8,False,True,False
9,False,True,False
10,False,True,False
11,False,True,False
12,False,True,False
13,False,True,False
14,False,True,False


# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

,A
1,6.000000e+66
2,6.000000e+66
3,6.000000e+66


In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

,objective_c,objective_d
9,-1.0,inf
3,-2.0,inf
4,-3.0,inf
5,-4.0,inf
